In [24]:
from twitterscraper import query_tweets
import datetime as dt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from time import time, sleep
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet
from sklearn.decomposition import LatentDirichletAllocation
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer

ModuleNotFoundError: No module named 'wordcloud'

In [13]:
NameOfCompanies = ["Apple", "Samsung", "Huawei"]
companies = []

In [14]:
i=0
while i < len(NameOfCompanies):   
    begin_date = dt.date(2019,11,1)
    end_date = dt.date(2020,2,1)
    limit = 0
    lang = "english"
    CompanyTweets = query_tweets(NameOfCompanies[i], limit=limit, begindate=begin_date, enddate=end_date, lang=lang)
    companies.append(CompanyTweets)
    i= i+1
else:
    print(NameOfCompanies[0])
    print(NameOfCompanies[1])
    print(NameOfCompanies[2])
    print("Finally finished!")

INFO: queries: ['Apple since:2019-11-01 until:2019-11-05', 'Apple since:2019-11-05 until:2019-11-10', 'Apple since:2019-11-10 until:2019-11-14', 'Apple since:2019-11-14 until:2019-11-19', 'Apple since:2019-11-19 until:2019-11-24', 'Apple since:2019-11-24 until:2019-11-28', 'Apple since:2019-11-28 until:2019-12-03', 'Apple since:2019-12-03 until:2019-12-07', 'Apple since:2019-12-07 until:2019-12-12', 'Apple since:2019-12-12 until:2019-12-17', 'Apple since:2019-12-17 until:2019-12-21', 'Apple since:2019-12-21 until:2019-12-26', 'Apple since:2019-12-26 until:2019-12-30', 'Apple since:2019-12-30 until:2020-01-04', 'Apple since:2020-01-04 until:2020-01-09', 'Apple since:2020-01-09 until:2020-01-13', 'Apple since:2020-01-13 until:2020-01-18', 'Apple since:2020-01-18 until:2020-01-22', 'Apple since:2020-01-22 until:2020-01-27', 'Apple since:2020-01-27 until:2020-02-01']
INFO: Got 19 tweets (19 new).
INFO: Got 37 tweets (18 new).
INFO: Got 56 tweets (19 new).
INFO: Got 76 tweets (20 new).
INFO

Apple
Samsung
Huawei
Finally finished!


In [15]:
len(companies)

3

In [16]:
dfNames = ['AppleTweets','SamsungTweets','HuaweiTweets']

In [17]:
i=0
for company in companies:
    dfNames[i] = pd.DataFrame(t.__dict__ for t in  company)
    i = i+1

In [18]:
AppleTweets = dfNames[0]
SamsungTweets = dfNames[1]
HuaweiTweets = dfNames[2]

In [19]:
AppleTweets.shape

(376, 21)

In [20]:
SamsungTweets.shape

(388, 21)

In [21]:
HuaweiTweets.shape

(388, 21)

In [ ]:
AppleTweets.head()

In [ ]:
SamsungTweets.head()

In [ ]:
HuaweiTweets.head()

In [ ]:
AppleTweets.columns

In [ ]:
SamsungTweetssungTweetsmsungTweets.columns

In [ ]:
HuaweiTweets.columns

In [ ]:
AppleTweets.drop(["screen_name", "username","user_id","tweet_id","tweet_url","timestamp_epochs",
                 "text_html","has_media","is_replied","is_reply_to","parent_tweet_id","reply_to_users"], axis = 1, inplace = True) 


In [ ]:
AppleTweets.head()

In [ ]:
SamsungTweets.drop(["screen_name", "username","user_id","tweet_id","tweet_url","timestamp_epochs",
                 "text_html","has_media","is_replied","is_reply_to","parent_tweet_id","reply_to_users"], axis = 1, inplace = True) 


In [ ]:
SamsungTweets.head()

In [ ]:
HuaweiTweets.drop(["screen_name", "username","user_id","tweet_id","tweet_url","timestamp_epochs",
                 "text_html","has_media","is_replied","is_reply_to","parent_tweet_id","reply_to_users"], axis = 1, inplace = True) 


In [ ]:
HuaweiTweets.head();

In [ ]:
# lemmatization with POS tag
def get_wordnet_pos(word):
    # map POS tag to the first character that lemmatize() accepts
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# clean and tokenize the text
def tokenize(text, stem=True, lemmatize=True, min_char=3, max_char=30):
    # tokenize, remove punctuations, digits  and make lowercase
    remove_map = dict((ord(char), ' ') for char in (string.digits+string.punctuation))
    tokens = nltk.word_tokenize(text.lower().translate(remove_map))
    tokens = [item for item in tokens if ((len(item)>=min_char)&(len(item)<=max_char))]

    # lemmatization
    if lemmatize:
        tokens = [WordNetLemmatizer().lemmatize(item, get_wordnet_pos(item)) for item in tokens]
    # stemming
    if stem:
        tokens = [PorterStemmer().stem(item) for item in tokens]

    return tokens
    
# vectorize text to either tfidf or bow
def vectorize(doc_list, method='tfidf', stem=True, lemmatize=True, **kwargs):
    # words that don't have much value and should not be included in the vectorization
    Additional_stop_words = ['will', 'laughter', 'applause', 'now']
    stop_words = text.ENGLISH_STOP_WORDS.union(Additional_stop_words)
    
    if method=='tfidf':
        vectorizer = TfidfVectorizer(tokenizer=lambda text: tokenize(text, stem=stem, lemmatize=lemmatize),
                                     stop_words=stop_words, **kwargs)
    
    # for simplicity I assume if not "tfidf", method is "bow" or "tf"
    else:
        vectorizer = CountVectorizer(tokenizer=lambda text: tokenize(text, stem=stem, lemmatize=lemmatize),
                                     stop_words=stop_words, **kwargs)
    
    vect = vectorizer.fit_transform(doc_list)
    
    return vect, vectorizer

In [ ]:
# plot the lenght of tweets given by each user for each company
def plot_word_counts(df, category='category', title=None, figsize=(12,6), suptitle_y=0.95): 
    fig, axs = plt.subplots(1, 2, figsize=figsize)
    plt.suptitle(title, fontsize=20, y=suptitle_y)

    # dictionary to map colors to each company
    my_palette = {'Apple': 'white', 'Samsung': 'blue','Huawei' : 'red'} 

    # violin plot to show the distribution of the word count
    sns.violinplot(x="word_count", y=category, hue="company",
                   data=df, split=True, palette=my_palette, ax=axs[0])
    axs[0].legend().set_visible(False)
    axs[0].set_ylabel('')

    # bar plot to show the average word count
    sns.barplot(x="word_count", y=category, hue="president",
                   data=df, palette=my_palette, ax=axs[1])
    axs[1].set_yticks([])
    axs[1].set_ylabel('')
    axs[1].set_xlabel('mean_word_count')

    plt.subplots_adjust(wspace=0.05)
    plt.show()

In [22]:
# plot word cloud of a given text
def plot_word_cloud(words, colormap='bwr', title=None, width=1000, height=800):
    # set the stop words
    stopwords = set(STOPWORDS)
    Additional_stop_words = ['will', 'laughter', 'applause', 'now']
    for s in  Additional_stop_words:
        stopwords.add(s)

    # generate WordCloud
    if type(words)==str:
        wc = WordCloud(background_color="white", stopwords=stopwords, 
                       width=width, height=height, colormap=colormap).generate(words)
    # for simplicity, I assume that if words is not an string, it is a dictionary of word frequencies
    else:
        wc = WordCloud(background_color="white", stopwords=stopwords, 
                       width=width, height=height, colormap=colormap).generate_from_frequencies(words)

    # show the WordCloud
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    if title:
        plt.title(title, fontsize=25, y=1.01)


# plot two word cloud, one for each president
def plot_two_word_clouds(df, title=None, method='str',**kwargs):
    my_color_map = ['Blues', 'Reds']
    plt.figure(figsize=(20, 10))
    for i, p in enumerate(['Barack Obama', 'George W. Bush']):
        if method=='str':
            words = (' '.join(df.loc[df['president']==p, 'content'])).lower()
        # for simplicity, I assume that if not string, the method is word frequency
        else:
            words = Counter(df.loc[df['president']==p, 'entity'])
            
        plt.subplot(1, 2, i+1)
        plot_word_cloud(words, my_color_map[i], title=p, **kwargs)

    if title:
        plt.suptitle(title, fontsize=35, y=0.9)
    
    plt.subplots_adjust(wspace=0.1)
    plt.show()

In [ ]:
# plot two separate pie cahrts of sentiment analysis results, one for each president
def plot_sentiment_pie(df, sent_label, title=None):
    counts = 100*(df.groupby([sent_label, 'president'])[sent_label].count() /
                  df.groupby(['president'])[sent_label].count())
    xs = np.array(counts).reshape(3, 2)
    
    
    plt.figure(figsize=(15, 8))
    for i, p in enumerate(['Barack Obama', 'George W. Bush']):    
        plt.subplot(1, 2, i+1)
        plt.pie(xs[:,i], explode=[0.07, 0.07, 0.07], labels=['Negative', 'Neutral', 'Positive'],
                colors=['#FD5D5F', '#FEDD64', '#38BF6F'], autopct='%.1f%%', shadow=True, textprops={'fontsize': 15})
        plt.axis('equal')
        plt.title(p, fontsize=20, y=0.94)
        
    if title:
        plt.suptitle(title, fontsize=25, y=0.96)
    
    plt.subplots_adjust(wspace=0.2)
    plt.show()

In [ ]:
# plot sentiment bar charts across all categories
def plot_sentiment_bars(df, sent_label, category, title=None):
    counts = 100*(df.groupby([sent_label, category])[sent_label].count() /
                  df.groupby([category])[sent_label].count())
    y = np.sort(df[category].unique())
    xs = np.array(counts).reshape(3, len(y))
    
    # positive bar
    p1 = plt.barh(y, xs[2], color='#38BF6F')
    # negative bar
    p2 = plt.barh(y, xs[0], left=xs[2], color='#FD5D5F')
    # neutral bar
    p3 = plt.barh(y, xs[1], left=xs[2]+xs[0], color='#FEDD64')

    plt.gca().invert_yaxis()
    plt.yticks(fontsize=15)
    plt.xlabel('Percentage of all sentences', fontsize=15)
    plt.legend((p1, p2, p3), ('Positive', 'Negative', 'Neutral'))
    if title:
        plt.title(title, fontsize=20)


# plot two separate sentiment bar charts, one for each president
def plot_two_sentiment_bars(df, sent_label, title=None, category='category_subcategory'):
    plt.figure(figsize=(12, 8))
    for i, p in enumerate(['Barack Obama', 'George W. Bush']):    
        plt.subplot(1, 2, i+1)
        plot_sentiment_bars(df.loc[df['president']==p], sent_label, category=category, title=p)
        if i==0:
            plt.legend().set_visible(False)
    
    plt.yticks([])
    if title:
        plt.suptitle(title, fontsize=30, y=1)
    
    plt.subplots_adjust(wspace=0.1)
    plt.show()

In [ ]:
# for ease of presentation, make a new column "category: subcategory"
data['category_subcategory'] = ['%s: %s' %(cat, subcat) for cat, subcat in
                                zip(data['category'].values, data['subcategory'].values)]

# count the number of words in each doc
word_count_list = np.zeros(data.shape[0], dtype=np.int)
doc_list = data['content'].values
for i in data.index:
    word_count_list[i] = len(doc_list[i].split())

# add the word count as a new column
data['word_count'] = word_count_list

# keep the memory clean
del word_count_list, doc_list
    
# check the df
print(data.shape)
data.head(2)

In [23]:
plot_two_word_clouds(AppleTweets, 'Word Cloud of All Apple')


NameError: name 'plt' is not defined

In [ ]:
plot_two_word_clouds(SamungTweets, 'Word Cloud of Samsung')


In [ ]:
plot_two_word_clouds(HuaweiTweets, 'Word Cloud of Huawei')


In [29]:
# 
# Named Entity Recognition
# Here comes our first bet. We can use NER to extract the named entities mentioned by each speacker. This could give us a better picture as to what they talked about the most.

# I am going to concentrate on 3 main categories of named entities, location, person and organisation.

# I am using a pretrained model from SpaCy. After doing some research on different pretrained NER options and their differences, I decided to use spaCy. It has a very good performance, even more than what we really need here. However, given more time, one could try to find / build an even more accurate NER model.

# Let's dive into NER.


In [ ]:
# this takes a few hours to run, I really suggest to load the saved results instead
if user_confirmation():
    t = time()
    
    # make a new df to store NER results
    columns=['sent_index', 'entity', 'label']
    sent_entities = pd.DataFrame(columns=columns)
    # convert df to array to improve loop time performance
    sentence_list = sent_data['sentence'].values
    # for every sentence, run NER and store the result
    for i in sent_data.index:
        sent = nlp(sentence_list[i])
        # to improve loop time performance, convert integer to string
        temp = [[str(i), x.text, x.label_] for x in sent.ents]
        # add 1 row for each NER result
        sent_entities = sent_entities.append(pd.DataFrame(temp, columns=columns), ignore_index=True)

    # convert sent_index back to integer
    sent_entities['sent_index'] = sent_entities['sent_index'].astype(np.int)

    print('Done in %i Seconds' %(time()-t))
    sent_entities.to_csv('data/sent_entities.csv', index=False)

    # keep the memory clean
    del sentence_list
else:
    t = time()
    # since it takes time to generate this, we load it from here
    sent_entities = pd.read_csv('data/sent_entities.csv')
    print('Done in %i Seconds' %(time()-t))

In [ ]:
# add other columns to sent_entities
sent_entities['president'] = sent_data.loc[sent_entities['sent_index'], ['president']].values

# check the df
print(sent_entities.shape)
sent_entities.head(2)